In [1]:
import os
import csv
from pathlib import Path
from functools import reduce
import time


In [2]:
os.getcwd()

'/opt/workspace'

In [3]:
path = Path('./AB_NYC_2019.csv').resolve()
path

PosixPath('/opt/workspace/AB_NYC_2019.csv')

In [4]:
with open(path, 'r', newline='', encoding='utf-8') as f:
    
    csv_file = csv.reader(f)
    
    row = 0
    for line in csv_file:
        row += 1
        print(line)
        if row > 5:
            break

['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
['2539', 'Clean & quiet apt home by the park', '2787', 'John', 'Brooklyn', 'Kensington', '40.64749', '-73.97237', 'Private room', '149', '1', '9', '2018-10-19', '0.21', '6', '365']
['2595', 'Skylit Midtown Castle', '2845', 'Jennifer', 'Manhattan', 'Midtown', '40.75362', '-73.98377', 'Entire home/apt', '225', '1', '45', '2019-05-21', '0.38', '2', '355']
['3647', 'THE VILLAGE OF HARLEM....NEW YORK !', '4632', 'Elisabeth', 'Manhattan', 'Harlem', '40.80902', '-73.9419', 'Private room', '150', '3', '0', '', '', '1', '365']
['3831', 'Cozy Entire Floor of Brownstone', '4869', 'LisaRoxanne', 'Brooklyn', 'Clinton Hill', '40.68514', '-73.95976', 'Entire home/apt', '89', '1', '270', '2019-07-05', '4.64', '1', '194']
['5022', 'Entire Apt: Spacious

In [5]:
def mapper_price(el: str) ->tuple([int]):
    try:
        price = int(el['price'])
        return (1, price, 0)
    except:
        return (0, 0, 0) 


def reducer(data1, data2) -> tuple([int, float]):
    if  data2[0] > 0:
        n = data1[0]
        n += data2[0]
        delta = data2[1] - data1[1]
        mean = data1[1] + delta / n
        M2 = data1[2] + delta * (data2[1] - mean)
        return n, mean, M2
    else:    
        return data1

In [17]:
%%timeit
with open(path, 'r', newline='', encoding='utf-8') as f:
    csv_file = csv.DictReader(f)
    n = 0
    mapped_data = map(mapper_price, csv_file)
#     print(list(mapped_data)[:10])
    n, mean, M2 = reduce(reducer, mapped_data)
    print(f'n = {n}, mean = {mean}, std_dev = {(M2 / n) ** (1/2)}, D = {M2/n}')

n = 48895, mean = 152.72068718682823, std_dev = 240.15171391941672, D = 57672.845698433375
n = 48895, mean = 152.72068718682823, std_dev = 240.15171391941672, D = 57672.845698433375
n = 48895, mean = 152.72068718682823, std_dev = 240.15171391941672, D = 57672.845698433375
n = 48895, mean = 152.72068718682823, std_dev = 240.15171391941672, D = 57672.845698433375
n = 48895, mean = 152.72068718682823, std_dev = 240.15171391941672, D = 57672.845698433375
n = 48895, mean = 152.72068718682823, std_dev = 240.15171391941672, D = 57672.845698433375
n = 48895, mean = 152.72068718682823, std_dev = 240.15171391941672, D = 57672.845698433375
n = 48895, mean = 152.72068718682823, std_dev = 240.15171391941672, D = 57672.845698433375
565 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
# len(prices)

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_json,col
from pyspark.sql.types import *

spark = SparkSession\
        .builder\
        .appName("pyspark-notebook")\
        .master("spark://spark-master:7077")\
        .config("spark.executor.memory", "512m")\
        .config("hive.metastore.uris", "thrift://hive-metastore:9083")\
        .config("spark.sql.warehouse.dir", "/user/hive/warehouse")\
        .enableHiveSupport()\
        .getOrCreate()


23/12/30 12:09:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [10]:
df_spark = spark.read.csv(path="./AB_NYC_2019.csv", sep=",", header=True)

In [11]:
df_spark.columns

['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [12]:
df1 = df_spark.select(df_spark.columns)

In [14]:
print(type(df_spark), type(df1))

<class 'pyspark.sql.dataframe.DataFrame'> <class 'pyspark.sql.dataframe.DataFrame'>


In [15]:
df_spark.write.saveAsTable('ab_nyc_2019')

23/12/30 12:24:02 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
